In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [3]:
url = 'https://jpmc.fa.oraclecloud.com/hcmUI/CandidateExperience/en/sites/CX_1001/requisitions?location=United+States&locationId=300000000289738&locationLevel=country'
driver = webdriver.Chrome()

driver.implicitly_wait(2)

driver.get(url)

In [4]:
next_button = driver.find_element('xpath', '//*[@class="search-results-load-more-btn"]')
while next_button:
    driver.execute_script("arguments[0].click();", next_button)
    time.sleep(2)
    try:
        next_button = driver.find_element('xpath', '//*[@class="search-results-load-more-btn"]')
    except:
        break
    

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@class="search-results-load-more-btn"]"}
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x00866643]
	(No symbol) [0x007FBE21]
	(No symbol) [0x006FDA9D]
	(No symbol) [0x00731342]
	(No symbol) [0x0073147B]
	(No symbol) [0x00768DC2]
	(No symbol) [0x0074FDC4]
	(No symbol) [0x00766B09]
	(No symbol) [0x0074FB76]
	(No symbol) [0x007249C1]
	(No symbol) [0x00725E5D]
	GetHandleVerifier [0x00ADA142+2497106]
	GetHandleVerifier [0x00B085D3+2686691]
	GetHandleVerifier [0x00B0BB9C+2700460]
	GetHandleVerifier [0x00913B10+635936]
	(No symbol) [0x00804A1F]
	(No symbol) [0x0080A418]
	(No symbol) [0x0080A505]
	(No symbol) [0x0081508B]
	BaseThreadInitThunk [0x75DD6BD9+25]
	RtlGetFullPathName_UEx [0x77A98FD2+1218]
	RtlGetFullPathName_UEx [0x77A98F9D+1165]


In [ ]:
job_title=[]
job_link=[]

soup = BeautifulSoup(driver.page_source, 'html.parser')
job_title.extend([td.text for td in soup.find_all("h3", {"class": "job-title"})])
job_link.extend([td['href'] for td in soup.find_all("a", {"class": "joblist-tile"})])  

In [ ]:
# create a dataframe that contains job titles and links for all job categories
df_title_link = pd.DataFrame(zip(job_title, job_link), columns=['JOB_TITLE', 'JOB_LINK'])

# drop the duplicates
df_title_link = df_title_link.drop_duplicates()

In [ ]:
# save the titles and links to csv
#df_title_link.to_csv('C:\Online_Courses\GitHub\Swatcloud\data\web_scraping\data\jpm_jobs_titles_links_all.csv')

## Extract job descriptions and qualifications for each job

In [ ]:
# load in the job titles and links from the csv file
#df_title_link = pd.read_csv('C:\Online_Courses\GitHub\Swatcloud\data\web_scraping\data\jpm_jobs_titles_links_all.csv', index_col=0)
#df_title_link.head()

In [ ]:
# retrieve job qualifications and descriptions
def job_description(title, link):
    qualifications = []
    description = []
    jobtitle = []
    joblink = []
    
    driver=webdriver.Chrome('chromedriver',options=chrome_options)
    for i in range(len(link)):
        URL=link[i]
        driver.get(URL)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        d = ''
              
        # get the job responsibilities
        try:
            tag = soup.find("div", {'class': "job-description"})
            tags = tag.find_all('ul')
            if len(tags) > 0:
                for t in tags:
                    d = d + " " + t.text
            #else:
                #d = tag.text
        except: pass
        
        description.append(d)   
        jobtitle.append(title[i])
        joblink.append(link[i])
        
    driver.quit()            
 
    return jobtitle, link, description, description

In [ ]:
# retrieve the qualification for each job.
title, link, qual, descrp = job_description(df_title_link['JOB_TITLE'].values, df_title_link['JOB_LINK'].values)

In [ ]:
def remove_space(s):
  s_new = s.replace('\n', ' ').replace('\xa0', ' ')
  while '  ' in s_new:
    s_new = s_new.replace('  ', ' ')
  return s_new

In [ ]:
qual_cleaned = [remove_space(q) for q in qual]
descrp_cleaned = [remove_space(d) for d in descrp]

In [ ]:
#create a dataframe from basic qualification and preferred qualification
df = pd.DataFrame(zip(title, qual_cleaned, link, descrp_cleaned))
df.columns = ['TITLE', 'QUALIFICATIONS', 'LINK', 'DESCRIPTION']
df['COMPANY'] = 'JPM'
df = df.iloc[:, [4, 0, 1, 2, 3]]

In [ ]:
# look for empty QUALIFICATION entries
def get_empty(df, colname) :
  empty_idx=[]
  for i in range(len(df[colname])):
    count = len(df[colname][i])
    if count<=10 : empty_idx.append(i)
    else: pass
  return empty_idx

empty_mini_q = get_empty(df, 'QUALIFICATIONS')
print(len(empty_mini_q))

# drop the empty Qualification entries
df.drop(empty_mini_q, inplace=True)

In [ ]:
#remove the duplicated jobs
df_nodup = df.drop_duplicates()
print("There are {} jobs from JP Morgan.".format(df_nodup.shape[0]))

df_nodup.to_csv('jpm_all_jobs_cleaned_short.csv')